In [1]:
import cv2
import numpy as np
import os
from scipy.interpolate import interp1d
from sklearn.decomposition import PCA
from pyefd import elliptic_fourier_descriptors
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [ ]:
def normalize_contour_points(contour):
    """Normalize contour points to a fixed number for consistency."""

    N = 250 # Number of points to normalize to
    contour_length = np.linspace(0, 1, len(contour)) # Length of the contour
    normalized_length = np.linspace(0, 1, N) # Normalized length
    interp_func_x = interp1d(contour_length, contour[:, 0], kind='linear') # Interpolation function for x
    interp_func_y = interp1d(contour_length, contour[:, 1], kind='linear') # Interpolation function for y
    normalized_contour = np.vstack((interp_func_x(normalized_length), interp_func_y(normalized_length))).T # Interpolated contour
    return normalized_contour

In [ ]:
def read_images_and_extract_contours(folder_path):
    """Read images from a folder and extract their largest external contour.

    Returns a list of normalized contour points for each image.
    """

    # Read the images from the folder
    images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    # Saving the contours of the images in the list
    contours_list = []

    for image_path in images:
        """
        1. Read the image
        2. Convert to grayscale
        3. Threshold the image
        4. Find the contours if any are present
        5. Get the largest contour
        6. Remove single dimensional entries from the contour
        7. Normalize the contour points and append to the list
        """
        image = cv2.imread(image_path) # Read the image 
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert to grayscale
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY) # Threshold the image
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # Find the contours
        if contours: # If contours are found
            contour = max(contours, key=cv2.contourArea) # Get the largest contour
            contour = contour.squeeze() # Remove single-dimensional entries from the contour
            contours_list.append(normalize_contour_points(contour)) # Normalize the contour points
    return contours_list

In [ ]:
def compute_fourier_coefficients(contour, order=30):
    """Compute Fourier coefficients for a given contour."""

    coeffs = elliptic_fourier_descriptors(contour, order=order, normalize=True) # Compute Fourier coefficients
    #return coeffs[1:]  # Skip the first coefficient as it's related to the image position
    return coeffs

In [ ]:
def compute_class_averages(fourier_descriptors, labels, n_clusters):
    """Compute average Fourier coefficients for each cluster."""
    
    sums = [np.zeros(fourier_descriptors[0].shape) for _ in range(n_clusters)]
    counts = [0] * n_clusters
    for coeffs, label in zip(fourier_descriptors, labels):
        sums[label] += coeffs
        counts[label] += 1
    averages = [sums[i] / counts[i] if counts[i] > 0 else None for i in range(n_clusters)]
    return averages